In [1]:
%reload_ext autoreload
%autoreload 2


# PROJET RI TESTS

In [4]:
import datasets
import utils
import features

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk


## DATA

In [ ]:
#T elechargement et sauvegarde des datasets 

# from datasets import load_dataset 
# wikiqa = load_dataset("wiki_qa")
# wikiqa.save_to_disk("wikiqa")

In [5]:
#Load le dataset ( enregistré en format pyarrow ./wikiqa )
wikiqa_data = datasets.load_from_disk("wikiqa")

In [6]:
print(wikiqa_data)
print(wikiqa_data["train"])

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 6165
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 2733
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 20360
    })
})
Dataset({
    features: ['question_id', 'question', 'document_title', 'answer', 'label'],
    num_rows: 20360
})


In [7]:
wikiqa_data_train = wikiqa_data["train"]    
questions = wikiqa_data_train["question"]   #questions
answers = wikiqa_data_train["answer"]       #answers
labels = wikiqa_data_train["label"]         #labels (pertinent non pertinent)

#Preprocessing ( see utils.preprocess )
questions = [utils.preprocess(txt) for txt in questions]
answers = [utils.preprocess(txt) for txt in answers]

In [9]:
# Example usage
from wikipedia2vec import Wikipedia2Vec
wiki2vec = Wikipedia2Vec.load('enwiki_20180420_500d.txt')
query = "example query"
document = "this is an example document used to demonstrate the Wikipedia2Vec similarity calculation"
print("Wikipedia2Vec similarity:", features.wikipedia2vec_similarity(wiki2vec, query, document))

KeyError: 52

### Overlap fraction between a query and a set of answers

In [20]:
# test de la feature overlap
qrep = ["apples banana"]
rep = ["I like apples and bananas.", "I prefer oranges."]

# Calculate overlap fraction feature for the sample data using the adapted function
overlap_fraction_features = list(features.calculate_overlap_fraction(qrep, rep))

print(overlap_fraction_features)

[autoreload of features failed: Traceback (most recent call last):
  File "C:\Users\NeilB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\NeilB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "d:\études\pldac\

ValueError: too many values to unpack (expected 2)

### Overlap synonym fraction between a query and a set of answers

In [43]:
# Sample data
qrep = "sad"
rep = "i'm sad and pitiful"

# Calculate overlap synonym fraction feature for the sample data using the function
overlap_syn_fraction_features = features.overlap_syn_fraction(qrep, rep)

print(overlap_syn_fraction_features)

print(utils.get_synonyms('sad'))

frere
len ['sad']
len de overlaf 2
frr
0.5
{'sorry', 'lamentable', 'pitiful', 'deplorable', 'distressing', 'sad'}


### TAGME : fraction of named entities in the query that are also present in the document.

In [60]:
query = "Barack Obama was the 44th President of the United States"
document = "Obama served as the 44th President"
print("TAGME overlap:", features.tagme_overlap(query, document))

query entitited {'44th', 'the United States', 'Barack Obama'}
TAGME overlap: 0.3333333333333333


### BM25 scores 

In [14]:
query = "query about bananas"
document = "documentary bla bla bla bananas"
corpus = [
    "documentary bla bla bla bananas",
    "this document is the second document without apples",
    "and this is the third one",
    "is this the first document but ananas are cool",
    "documentary bla bla bla"
]

score = features.bm25_score(query, document, corpus)
print(f"BM25 score: {score}")


BM25 score: 1.5376575235991854


## WORD2VEC SIMILARITY

In [16]:
# Exemple d'utilisation
query = "example query"
document = "this is an example document used to demonstrate word2vec similarity"
print("Word2Vec similarity:", features.word2vec_similarity(query, document))


Word2Vec similarity: 0.46693054


In [ ]:
query = "example query"
document = "query example"
print("Word2Vec similarity:", word2vec_similarity(query, document))

Word2Vec similarity: 1.0000001


Word2Vec similarity: 1.0000001


In [ ]:
import nltk
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.lm.preprocessing import pad_both_ends

nltk.download('gutenberg')
from nltk.corpus import gutenberg

# Train a unigram language model using Gutenberg corpus
text = gutenberg.raw('shakespeare-hamlet.txt')
tokens = text.split()
train_data = [list(ngrams(pad_both_ends(tokens, n=1), n=1)) for n in range(1, 2)]
lm = Laplace(1)  # Laplace smoothing with n=1
lm.fit(train_data)

def calculate_lm_score(text):
    test_data = list(ngrams(pad_both_ends(text.split(), n=1), 1))
    score = lm.perplexity(test_data)
    return score

# Sample data
text1 = "To be or not to be"
text2 = "To sleep, perchance to dream"

# Calculate LM score for the sample data
lm_score1 = calculate_lm_score(text1)
lm_score2 = calculate_lm_score(text2)

print("LM score for text1:", lm_score1)
print("LM score for text2:", lm_score2)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\NeilB\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


ValueError: Cannot fit without a vocabulary or text to create it from.